<a href="https://colab.research.google.com/github/Swastik200/30DaysOfDSA/blob/main/Adaboost2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!cp "/content/drive/MyDrive/CMAPSS.zip" "/content"
!unzip CMAPSS.zip

Archive:  CMAPSS.zip
  inflating: CMAPSS/readme.txt       
  inflating: CMAPSS/RUL_FD001.txt    
  inflating: CMAPSS/RUL_FD002.txt    
  inflating: CMAPSS/RUL_FD003.txt    
  inflating: CMAPSS/RUL_FD004.txt    
  inflating: CMAPSS/test_FD001.txt   
  inflating: CMAPSS/test_FD002.txt   
  inflating: CMAPSS/test_FD003.txt   
  inflating: CMAPSS/test_FD004.txt   
  inflating: CMAPSS/train_FD001.txt  
  inflating: CMAPSS/train_FD002.txt  
  inflating: CMAPSS/train_FD003.txt  
  inflating: CMAPSS/train_FD004.txt  
  inflating: CMAPSS/x.txt            


In [4]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from math import sqrt

In [5]:
index_names = ['unit_number', 'time_cycles']
setting_names = ['setting_1', 'setting_2', 'setting_3']
sensor_names = ['sensor_{}'.format(i) for i in range(1,22)]
col_names = index_names + setting_names + sensor_names

In [6]:
path='/content/CMAPSS/'
train_df = pd.read_csv(path+'train_FD002.txt', delimiter=' ', header=None)
test_df = pd.read_csv(path+'test_FD002.txt', delimiter=' ', header=None)
columns = ['unit_number', 'time_in_cycles', 'operational_setting_1', 'operational_setting_2', 'operational_setting_3'] + [f'sensor_{i}' for i in range(1, 24)]
train_df.columns = columns
test_df.columns=columns

In [7]:
max_cycles = train_df.groupby('unit_number')['time_in_cycles'].max().reset_index()
max_cycles.columns = ['unit_number', 'max_cycles']

In [8]:
max_cycles = test_df.groupby('unit_number')['time_in_cycles'].max().reset_index()
max_cycles.columns = ['unit_number', 'max_cycles']

In [9]:
train_df = train_df.merge(max_cycles, on='unit_number', how='left')
train_df['RUL'] = train_df['max_cycles'] - train_df['time_in_cycles']
train_df.drop('max_cycles', axis=1, inplace=True)

In [10]:
test_df = test_df.merge(max_cycles, on='unit_number', how='left')
test_df['RUL'] = test_df['max_cycles'] - test_df['time_in_cycles']
test_df.drop('max_cycles', axis=1, inplace=True)

In [11]:
features = ['operational_setting_1', 'operational_setting_2', 'operational_setting_3'] + [f'sensor_{i}' for i in range(1, 22)]
target = 'RUL'

In [12]:
adaboost = AdaBoostRegressor(n_estimators=50, random_state=42)
adaboost.fit(test_df[features], test_df[target])

AdaBoostRegressor(random_state=42)

In [13]:
train_pred = adaboost.predict(train_df[features])
test_pred = adaboost.predict(test_df[features])

In [16]:
import numpy as np
from sklearn.metrics import mean_squared_error

# Remove NaN values from train_df[target] and train_pred
train_df_clean = train_df.dropna(subset=[target])
train_pred_clean = train_pred[~np.isnan(train_df[target])]

# Calculate RMSE
train_rmse = np.sqrt(mean_squared_error(train_df_clean[target], train_pred_clean))
print(f"Train RMSE: {train_rmse}")


Train RMSE: 114.93271332338652


In [17]:
test_rmse = sqrt(mean_squared_error(test_df[target], test_pred))
print(f"Test RMSE: {test_rmse}")

Test RMSE: 60.90519080416483
